# Smart Complaint deduplication using Snowflake-native AISQL

By Anant Damle & Nathan Birch

Story Link: [https://medium.com/snowflake/smart-complaint-deduplication-using-snowflake-native-aisql-2bab5885e277](https://medium.com/snowflake/smart-complaint-deduplication-using-snowflake-native-aisql-2bab5885e277)

Customer issue management is an inevitable part of doing business in today’s world. Duplicate complaints remain a common
challenge for organisations, often stemming from customers following up on similar issues through different channels and
lack of reliable master account information, leading to frustrating experiences for both customers and support teams,
and ultimately, skewed reporting.

Imagine a customer raising a complaint via a webform, then following up with a phone call about the same issue.
Without a robust system, the on-call person might be unaware of the previous interaction, leading to redundant efforts
and a lack of visibility into the true state of a customer's issue. The Challenge: unreliable matching, lost visibility
and wasted resources.

Traditional methods often rely on name/entity resolution, extracting attributes like name, phone number, or email using
regex or other forms of patterns and then match to related issues. However, these identifiers aren't always reliable.
Account numbers or License Plate Numbers (LPNs) can be inconsistent, and even within a short timeframe (e.g., 5 days),
the same issue might be treated as a new complaint. The result? Duplicate reports that obscure the real count of unique
issues, hindering effective analysis and decision-making.


# Architecture

Run the following python cell to display the architecture diagram

In [ ]:
import streamlit as st

st.image("architecture.jpg")

A Snowflake-native solution designed to address these deduplication challenges, provides a clear and accurate picture of
customer complaints.

1. **Preprocessing/Enrichment:** Extract entities and summarise the case using Snowflake Cortex  
2. **Master Entity Resolution:** Associate cases to an account id through a combination of deterministic, fuzzy and
   vector matching techniques.  
3. **Similarity Analysis & Deduplication:** Compares cases within the same master account to generate a similarity score
   and explanation, ensuring efficient and accurate deduplication.  
4. **Orchestration & Automation:** You can automatically update CRM from Snowflake using stored procedures or a 
   Streamlit based dashboard to visualise the results. These will need to be built and are not included in the code 
   artifacts.


# Objectives
Key Snowflake Functions Covered:

[`AI_COMPLETE`](https://docs.snowflake.com/en/sql-reference/functions/ai_complete) – Generates structured data from unstructured text using prompts, and provides content similarity
analysis with explanations.

[`AI_EMBED`](https://docs.snowflake.com/en/sql-reference/functions/ai_embed) – Converts text into vector embeddings, enabling semantic search and matching based on meaning.

[`VECTOR_COSINE_SIMILARITY`](https://docs.snowflake.com/en/sql-reference/functions/vector_cosine_similarity) – Compares vectors generated by AI\_EMBED to identify related content through semantic
similarity.

[`JAROWINKLER_SIMILARITY`](https://docs.snowflake.com/en/sql-reference/functions/jarowinkler_similarity) – Performs fuzzy string matching, useful for handling names or identifiers with typos
or slight variations.


# Costs

This solution uses Snowflake Cortex AISQL functions which incur compute costs based on the number of tokens processed.
Refer to the [details](https://docs.snowflake.com/user-guide/snowflake-cortex/aisql?lang=de%2F#cost-considerations) for each function’s cost in credits.

# Before you begin

Create a [Snowflake notebook](https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks) from the Git repository
using these [steps](https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks-snowgit#create-a-notebook-from-a-file-in-a-git-repository).

*
   > NOTE: Alternatively to run the notebook locally follow the instructions
   > in the README file to convert this notebook to a normal Jupyter notebook
   > with snowsql cell magic.

# Solution

Follow the instructions to setup the data and then use AISQL functions to identify similar cases.


## Pre-Setup

Create role, database, schemas, warehouse and then `GRANT` permissions to the new role with `SNOWFLAKE_CORTEX` permissions to be able to use AISQL functions.

1. Create a separate role and database for running this demo.

    > NOTE: If you do not have `ACCOUNTADMIN` role, 
    > kindly request a new role and database to be created
    > before following to next steps

In [ ]:
SET current_user_name = (SELECT CURRENT_USER());
SET demo_role_name = 'CASE_DEDUP_DEMO_ROLE';
SET demo_db_name = 'CASE_DEDUP_DEMO_DB';
SET big_llm_model = 'mistral-large2';
SET vectorization_model = 'snowflake-arctic-embed-l-v2.0-8k';

In [7]:
USE ROLE ACCOUNTADMIN;
-- create demo role
CREATE OR REPLACE ROLE IDENTIFIER($demo_role_name);

-- grant role to current user
GRANT ROLE IDENTIFIER($demo_role_name) TO USER IDENTIFIER($current_user_name);

-- grant cortex role to demo role to allow using AISQL functions
GRANT DATABASE ROLE SNOWFLAKE.CORTEX_USER TO ROLE IDENTIFIER($demo_role_name);

-- create demo database
CREATE OR REPLACE DATABASE IDENTIFIER($demo_db_name);

-- grant ownership to demo role
GRANT OWNERSHIP ON DATABASE IDENTIFIER($demo_db_name) TO ROLE IDENTIFIER($demo_role_name);

Query executed successfully, but it did not produce any results to display.


2. Start using the demo role and create schemas

In [ ]:
-- switch to demo role
USE ROLE IDENTIFIER($demo_role_name);

USE DATABASE IDENTIFIER($demo_db_name);

In [ ]:
-- create raw schema
CREATE OR REPLACE SCHEMA RAW;

-- create enriched schema
CREATE OR REPLACE ENRICHED;

-- create presentation schema
CREATE OR REPLACE SCHEMA PRESENTATION;

3. Populate sample data for support interactions and account entities table.

   There are 2 tables that need to be imported from the case management system to Snowflake, 
   1. The `raw_support_interactions` table contains the user/agent interactions for each case, a real system would       have many dimensions like master_account_id, case_channel, etc. This demo only expects two columns to
      demonstrate entity extraction process.

   2. The `raw_master_identities` table contains a list of identities associated with each master account, like,         Vehicle plate number, phone number, email address, name, etc.

In [ ]:
USE SCHEMA RAW;

CREATE OR REPLACE TABLE raw_support_interactions (
  case_id STRING,
  transcript STRING
);

CREATE OR REPLACE  TABLE raw_master_identities (
  account_id STRING,
  type STRING,
  value STRING
);

-- CREATE FILE FORMAT
CREATE OR REPLACE TEMP FILE FORMAT CSV_GZ
TYPE = CSV
FIELD_DELIMITER = ','
SKIP_HEADER = 1
FIELD_OPTIONALLY_ENCLOSED_BY = '"'
EMPTY_FIELD_AS_NULL = TRUE
TRIM_SPACE = TRUE
ENCODING = 'UTF8';

In [ ]:
# Load csv files into the tables' internal stages
from snowflake.snowpark.context import get_active_session

session = get_active_session()

_ = session.file.put(
    "raw_interactions_transcript.csv",
    "@CASE_DEDUP_DEMO_DB.RAW.%raw_support_interactions",
)
_ = session.file.put(
    "master_identities.csv", "@CASE_DEDUP_DEMO_DB.RAW.%raw_master_identities"
)

In [ ]:
-- Load transcripts into the table
COPY INTO raw_support_interactions
FILE_FORMAT = CSV_GZ;

-- See the transcripts
SELECT * FROM raw_support_interactions;

In [ ]:
-- load master identities into the table
COPY INTO raw_master_identities
FILE_FORMAT = CSV_GZ;

SELECT * FROM raw_master_identities;

# Phase 1: Data enrichment / Preprocessing

The initial step involves extracting entities from raw support transcripts and summarizing them. This optimizes costs
by reducing the number of tokens required for vectorization but also creates a valuable knowledge base for quick search
capabilities. The solution leverages Snowflake Cortex's [`AI_COMPLETE`](https://docs.snowflake.com/en/sql-reference/functions/ai_complete) function to extract specific information
(sentiment, summary, contacts) from unstructured text, even for entities that don't follow strict patterns.
This enriched data is then vectorized using [`AI_EMBED`](https://docs.snowflake.com/en/sql-reference/functions/ai_embed) for efficient matching.

`AI_COMPLETE` provides a mechanism for specifying the [LLM model](https://docs.snowflake.com/en/sql-reference/functions/ai_complete-single-string#arguments) that one wants to use and extracting structured
data.


1. Create enriched data tables

In [ ]:
USE SCHEMA ENRICHED;

CREATE OR REPLACE TABLE enriched_support_interactions (
  case_id STRING,
  transcript STRING,
  enrichment VARIANT,
  transcript_summary_vector VECTOR(FLOAT, 1024) -- can be used as knowledge base
);

CREATE OR REPLACE TABLE enriched_master_identities (
  account_id VARCHAR,
  type VARCHAR,
  value VARCHAR,
  value_vector VECTOR(FLOAT, 1024)
);

2. Extract entities from case transcripts and update the `enriched_support_interactions` table.

In [ ]:
INSERT INTO enriched_support_interactions
WITH entity_and_summary AS (
  SELECT
    case_id,
    transcript,
    AI_COMPLETE(
      model => $big_llm,
      prompt => CONCAT(
        'Extract this transcript''s summary, sentiment, and contacts including contact_type oneof (name, company_name, phone_number, email, vehicle_registration):',
        transcript
      ),
      response_format => {
  'type': 'json',
  'schema': {
    'type': 'object',
    'properties': {
      'sentiment': {
        'type': 'string',
        'description': 'The sentiment of the interaction.'
      },
      'summary': {
        'type': 'string',
        'description': 'A summary of the interaction.'
      },
      'contacts': {
        'type': 'array',
        'description': 'A summary of the contacts.',
        'items': {
          'type': 'object',
          'description': 'A contact information like phone numner, name, email address, or company name',
          'properties': {
            'type': {
              'type': 'string',
              'description': 'the type of contact - email, phone_number, name, company_name, vehicle_registration'
            },
            'value': {
              'type': 'string',
              'description': 'the value of the contact'
            }
          },
          'required': [
            'type',
            'value'
          ]
        }
      }
    },
    'required': [
      'sentiment',
      'summary',
      'contacts'
    ]
  }
}
)::VARIANT as enrichment
  FROM
    RAW.raw_support_interactions
)
SELECT
  case_id,
  transcript,
  OBJECT_CONSTRUCT(
    'sentiment',
    enrichment:sentinent::STRING,
    'summary',
    enrichment:summary::STRING,
    'contacts',
    TRANSFORM(
      enrichment:contacts::ARRAY(VARIANT),
      c VARIANT -> OBJECT_CONSTRUCT(
        'type',
        c:type::STRING,
        'value',
        CASE
          c:type
          WHEN 'phone_number'
            THEN REGEXP_REPLACE(c:value::STRING, '[^0-9]', '')
          WHEN 'vehicle_registration'
            THEN REGEXP_REPLACE(c:value::STRING, '[^0-9A-Za-z]', '')
          ELSE c:value::STRING
        END
      )
    )::VARIANT
  ) AS enrichment,
  AI_EMBED(
    $vectorization_model,
    enrichment:summary::STRING
  ) AS transcript_summary_vector
FROM entity_and_summary;


-- visualise the enriched support interactions
SELECT * EXCLUDE (transcript_summary_vector) FROM enriched_support_interactions;

3. Vectorise the name like values in raw identities table

In [ ]:
INSERT INTO enriched_master_identities
SELECT
  account_id,
  type,
  value,
  IFF(
    type IN ('name', 'company_name'),
    AI_EMBED($vectorization_model, value),
    NULL
  ) AS value_vector
FROM RAW.raw_master_identities;

SELECT * FROM enriched_master_identities;

## Phase 2: Master Entity Resolution

In this stage, cases that do not have preidentified master account ids are probabilistically linked to a master account
ID using identified case entities. 

The first part of this stage combines deterministic matching (exact matches for phone, email, vehicle registration)
with fuzzy matching using [`JAROWINKLER_SIMILARITY`](https://docs.snowflake.com/en/sql-reference/functions/jarowinkler_similarity), you can use any other similarity function in conjunction
with similarity function(example [SOUNDEX](https://docs.snowflake.com/en/sql-reference/functions/soundex_p123)) depending on your needs.

Finally a vector matching with [`VECTOR_COSINE_SIMILARITY`](https://docs.snowflake.com/en/sql-reference/functions/vector_cosine_similarity) for names and company names.
The goal is to accurately associate all related interactions with a single customer profile,
even with variations in data entry.

In [17]:
-- store the case to master account mapping (still in ENRICHED schema)
CREATE OR REPLACE TABLE case_master_map (case_id STRING, master_account_id STRING);

Query executed successfully, but it did not produce any results to display.


In [18]:
INSERT INTO case_master_map
WITH
  flattened_cases AS (
    SELECT
      case_id,
      c.value:type::STRING AS id_type,
      c.value:value::STRING AS id,
      IFF (
        c.value:type::STRING IN ('name', 'company_name'),
        AI_EMBED($vectorization_model, c.value:value::STRING),
        NULL
      ) AS id_vector
    FROM
      enriched_support_interactions,
      LATERAL FLATTEN(ENRICHMENT:contacts) AS c
  ),
  combined_matches AS (
    SELECT
      f.case_id,
      f.id_type,
      f.id,
      m.account_id AS master_account_id,
      IFF(
        f.id_type IN ('name', 'company_name'),
        ((
          JAROWINKLER_SIMILARITY(f.id, m.value) +
          VECTOR_COSINE_SIMILARITY(f.id_vector, m.value_vector) * 100
        )/2),
        JAROWINKLER_SIMILARITY(f.id, m.value)
      ) AS similarity_score
    FROM
      flattened_cases f
      LEFT OUTER JOIN enriched_master_identities m ON f.id_type = m.type
      AND (
        -- exact match
        (
          f.id_type IN ('phone_number', 'email', 'vehicle_registration')
          AND f.id = m.value
        )
        OR
        -- fuzzy name match using Jaro-winkler match
        (
          f.id_type IN ('name', 'company_name')
          AND JAROWINKLER_SIMILARITY(f.id, m.value) >= 85
        )
        OR
        -- vector matching the name
        (
          f.id_type IN ('name', 'company_name')
          AND VECTOR_COSINE_SIMILARITY(f.id_vector, m.value_vector) > 0.8
        )
      )
  )
-- The second part of this stage then assigns a single master account ID to a case 
-- based on the master ID that has the highest match ratio, cases that do not match
-- existing master account information are separated and will be surfaced as possible 
-- new accounts.
SELECT
  case_id,
  master_account_id
FROM combined_matches
GROUP BY case_id, master_account_id
QUALIFY ROW_NUMBER() OVER (PARTITION BY case_id ORDER BY AVG(similarity_score) DESC) = 1
ORDER BY case_id;

Number of rows inserted: 10


In [ ]:
-- Visualise the results
SELECT * FROM case_master_map;

## Phase 3: Case deduplication through similarity analysis

Once cases are linked to a master account, a self-join is performed within cases of the same master id using
[`AI_COMPLETE`](https://docs.snowflake.com/en/sql-reference/functions/ai_complete) to compare cases using a prompt that
generates a similarity score (between 0 and 1\) and an explanation for the relationship between two cases. This process
is optimized to limit the comparisons to cases within the same master account, ensuring efficiency and accuracy.

The results will be stored in `PRESENTATION` schema.

In [20]:
-- The deduplicated results should be stored in `PRESENTATION` schema.
USE SCHEMA PRESENTATION;

CREATE OR REPLACE TABLE duplicate_cases (
  case_a VARCHAR,
  case_b VARCHAR,
  master_account_id VARCHAR,
  similarity_score FLOAT,
  similarity_explanation VARCHAR
);

Query executed successfully, but it did not produce any results to display.


Run the deduplication query and insert the results into the `duplicate_cases` table.

In [21]:
INSERT INTO duplicate_cases (case_a, case_b, master_account_id, similarity_score, similarity_explanation)
WITH  
  mapped_support_cases AS (
    SELECT
      s.case_id,
      m.master_account_id,
      s.enrichment:summary::STRING as transcript_summary
    FROM
      ENRICHED.enriched_support_interactions s
      LEFT JOIN ENRICHED.case_master_map m ON s.case_id = m.case_id
      -- Omit cases that are not mapped to a master account for similarity analysis
      WHERE m.master_account_id IS NOT NULL
  ),
  cross_joined_cases AS (
    SELECT
      t1.master_account_id,
      t1.case_id AS case_a,
      t2.case_id AS case_b,
      t1.transcript_summary AS transcript_summary_a,
      t2.transcript_summary AS transcript_summary_b,
      AI_COMPLETE(
        model => $big_llm_model,
        prompt => CONCAT(
          'Summarise why the following interactions are related and return a similarity score on a scale of 0 to 1 both inclusive, where 1 means the interactions are identical.

          interaction 1 is:\n',
          transcript_summary_a,
          '\ninteraction 2 is:',
          transcript_summary_b
        ),
        response_format => {
    'type': 'json',
    'schema': {
      'type': 'object',
      'properties': {
        'similarity_score': {
          'type': 'number',
          'description': 'The similarity score of interactions on a scale of 0 to 1 both inclusive.'
        },
        'similarity_explanation': {
          'type': 'string',
          'description': 'reason for the score and summary of the similarity between interactions.'
        }
      },
      'required' : ['similarity_score', 'similarity_explanation']
    }
})::VARIANT AS summary_similarity_analysis
    FROM
      mapped_support_cases t1
      JOIN
      mapped_support_cases t2
      -- ensure cartesian product only within a master-account's cases
      ON t1.master_account_id = t2.master_account_id
      -- Remove duplicate pairs
      -- (e.g., (A,B) but not (B,A)).
      AND t1.case_id < t2.case_id
  )
SELECT
  case_a,
  case_b,
  master_account_id,
  summary_similarity_analysis:similarity_score::FLOAT AS similarity_score,
  summary_similarity_analysis:similarity_explanation::STRING AS similarity_explanation
FROM cross_joined_cases
WHERE summary_similarity_analysis:similarity_score::FLOAT > 0.8;

Number of rows inserted: 4


### Visualise the results

In [22]:
SELECT * FROM duplicate_cases ORDER BY case_a;

Result stored in DataFrame 'duplicate_cases_df'.


,CASE_A,CASE_B,MASTER_ACCOUNT_ID,SIMILARITY_SCORE,SIMILARITY_EXPLANATION
0,CASE_1001,CASE_1002,USER_ALPHA,0.95,"Both interactions involve Samantha reporting a problem with the same order number (A452). In both cases, the issue is related to the device not charging properly. Additionally, both interactions imply that Samantha is seeking a resolution or advice for the problem."
1,CASE_1001,CASE_1007,USER_ALPHA,0.90,"Both interactions involve Samantha reporting a problem with a device not charging properly. In the first interaction, she specifically mentions order #A452 and requests advice on a replacement. In the second interaction, she follows up on a previous case with the same issue, and the agent confirms that a replacement will be sent as the device is under warranty. The core issue (device not charging) and the requested resolution (replacement) are the same in both interactions."
2,CASE_1002,CASE_1007,USER_ALPHA,0.90,"Both interactions involve Samantha contacting support regarding a device that won't hold charge. The first interaction is about reporting the issue, while the second is a follow-up on the same issue, with a confirmation that the device is under warranty and will be replaced. The core issue (device not holding charge) is the same in both interactions."
3,CASE_1004,CASE_1005,USER_BETA,0.90,"Both interactions are related to the same insurance claim for hailstorm damage to a vehicle with the registration XYZ-789. They involve the same customer and the same claim number (implicitly in the second interaction). The second interaction is a follow-up to the first, with the customer sending the photos they mentioned in the initial call and inquiring about next steps."


## Phase 3.1 Identify potential new accounts

Entity extraction from transcripts can identify new contacts which can be used to generate new accounts.

In [23]:
WITH
  cases_without_master_accounts AS (
    SELECT 
      case_id
    FROM ENRICHED.case_master_map
    WHERE master_account_id IS NULL
)
  SELECT
    UUID_STRING('fe971b24-9572-4005-b22f-351e9c09274d', cwm.case_id) AS new_master_account_id,
    cwm.case_id,
    es.enrichment:contacts as contacts
  FROM    
    cases_without_master_accounts cwm
    LEFT JOIN ENRICHED.enriched_support_interactions es 
    ON 
      cwm.case_id = es.case_id    
      WHERE es.enrichment:contacts IS NOT NULL;

Result stored in DataFrame 'new_master_accounts'.


,NEW_MASTER_ACCOUNT_ID,CASE_ID,CONTACTS
0,4c90545c-118d-5a0f-8515-673af6fc15d0,CASE_1008,"[\n {\n ""type"": ""phone_number"",\n ""value"": ""0469100987""\n },\n {\n ""type"": ""name"",\n ""value"": ""Andy""\n }\n]"
1,b9497fd6-9509-5b55-88ba-84581f382767,CASE_0999,"[\n {\n ""type"": ""phone_number"",\n ""value"": ""0469100787""\n },\n {\n ""type"": ""name"",\n ""value"": ""Andrew""\n }\n]"
2,0216f9f0-91de-528c-8ea7-5515ff98a055,CASE_997,"[\n {\n ""type"": ""phone_number"",\n ""value"": ""0469100987""\n },\n {\n ""type"": ""name"",\n ""value"": ""Andy""\n }\n]"


# Takeaways

The solutions delivers:

* Duplicate cases table with four key columns:

    * **Parent CaseID:** The prior case ID.  
    * **Child CaseID:** The next message related to the parent or the same issue.  
    * **Measure of Confidence:** A probability score indicating how related the cases are.  
    * **Explanation:** A clear reason why the cases are related.

* potential new accounts: providing visibility into potential missing CRM entries.

* A structured output that provides immediate clarity, 
  allowing customer support to report accurate complaint counts and 
  gain better visibility into ongoing issues, Customer Feedback and Future Directions.

* allays performance concerns by prunning the cross-join scenario by limit the joins within an account's cases.

This solution showcases that Snowflake provides a platform that facilitates both efficient
and seamless data processing for AI use-cases. 

You can benefit by leveraging Snowflake's powerful capabilities, to transform complaints management unstructured data,
by moving beyond fragmented data to a unified, intelligent approach that benefits both the organisation and theircustomers.

# Next Steps

Use this [notebook](https://github.com/Snowflake-Labs/sfguide-customer-issue-deduplication-demo/blob/main/ISSUE_DEDUP_NOTEBOOK.ipynb) to run a demo or adapt to your own environment for a Proof of Concept (POC)

# Disclaimer
License: Apache 2.0

This is not an official Snowflake product or feature.